In [4]:
!pip install selenium
!pip install webdriver-manager
!pip install tqdm
# Google Colab에 필요한 라이브러리 설치
!apt-get update -qq
!apt-get install -qq chromium-chromedriver
!pip install selenium tqdm webdriver-manager

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [5]:
# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import numpy as np
import chromedriver_autoinstaller  # setup chrome options


chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

chromedriver_autoinstaller.install()  # set the target URL

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.
cp: cannot create regular file '/content/drive/MyDrive/Colab Notebooks': No such file or directory


In [6]:

# colab 환경에서 selenium을 사용하기 위한 환경 설정
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

# Google Colab 환경에 맞게 Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # GUI 없는 모드
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--disable-gpu')  # GPU 비활성화
options.add_argument('--window-size=1920x1080')
options.add_argument('--disable-software-rasterizer')
options.add_argument('--remote-debugging-port=9222')
options.add_argument('--start-maximized')
options.add_argument('--user-data-dir=/tmp/user_data')  # User data dir 설정

# 크롬 드라이버 설정 및 실행
driver = webdriver.Chrome(options=chrome_options)

# 뉴스 크롤링 함수: 각 뉴스 URL에서 제목, 날짜, 본문 크롤링
def news_scraping(news_url, driver):
    driver.get(news_url)
    try:
        # 기사 제목 로드 대기
        title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'media_end_head_headline'))
        ).text
        # 날짜 로드 대기
        datetime = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span'))
        ).text
        # 본문 로드 대기
        article = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'dic_area'))
        ).text
    except Exception as e:
        print(f"크롤링 중 에러 발생: {e}")
        title, datetime, article = None, None, None

    return [title, datetime, article, news_url]

# 스크롤을 더 이상 할 수 없을 때까지 뉴스 타이틀을 추가로 로드하는 함수
def scroll_until_no_more_content(driver, pause_time=2):
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 페이지 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)  # 스크롤 후 대기

        # 새로운 페이지 높이 확인
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:  # 스크롤할 내용이 더 이상 없으면 중단
            break

        last_height = new_height

# 네이버 뉴스 검색 페이지 열기
driver.get("https://search.naver.com/search.naver?where=news&query=%ED%98%84%EB%8C%80%EC%B0%A8&sm=tab_opt&sort=1&photo=0&field=0&pd=3&ds=2024.06.16&de=2024.06.17&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Add%2Cp%3Afrom20240616to20240617&is_sug_officeid=0&office_category=0&service_area=0")

# 스크롤을 더 이상 할 수 없을 때까지 뉴스 로드
scroll_until_no_more_content(driver, pause_time=2)

# 네이버 뉴스 링크들만 추출
link_elements = driver.find_elements(By.PARTIAL_LINK_TEXT, '네이버뉴스')
news_links = [elem.get_attribute('href') for elem in link_elements]

print(f"{len(news_links)}개의 뉴스 링크를 찾았습니다.")

# CSV 파일로 저장할 데이터 리스트
news_data = []

# 각 뉴스 링크에 들어가서 제목, 날짜, 본문 크롤링
for news_link in tqdm(news_links, desc="뉴스 크롤링 진행 중", ncols=100):  # tqdm 추가
    news_info = news_scraping(news_link, driver)
    if news_info[0]:  # 제목이 있는 경우만 저장 (크롤링 실패시 None 처리됨)
        news_data.append(news_info)

# 드라이버 종료
driver.quit()

# 크롤링한 데이터를 CSV 파일로 저장
csv_filename = '1617.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Date", "Article", "URL"])  # CSV 파일 헤더
    writer.writerows(news_data)

print(f"크롤링한 데이터를 {csv_filename} 파일에 저장했습니다.")

364개의 뉴스 링크를 찾았습니다.


뉴스 크롤링 진행 중:  48%|███████████████████                     | 174/364 [07:20<17:22,  5.49s/it]

크롤링 중 에러 발생: Message: 
Stacktrace:
#0 0x5c211d5084ca <unknown>
#1 0x5c211d01b620 <unknown>
#2 0x5c211d06a306 <unknown>
#3 0x5c211d06a5a1 <unknown>
#4 0x5c211d0afa04 <unknown>
#5 0x5c211d08e39d <unknown>
#6 0x5c211d0acde0 <unknown>
#7 0x5c211d08e113 <unknown>
#8 0x5c211d05cbe0 <unknown>
#9 0x5c211d05dbbe <unknown>
#10 0x5c211d4d4e4b <unknown>
#11 0x5c211d4d8de2 <unknown>
#12 0x5c211d4c1d2c <unknown>
#13 0x5c211d4d9957 <unknown>
#14 0x5c211d4a74bf <unknown>
#15 0x5c211d4f7348 <unknown>
#16 0x5c211d4f7510 <unknown>
#17 0x5c211d507346 <unknown>
#18 0x7b52deda2ac3 <unknown>



뉴스 크롤링 진행 중: 100%|████████████████████████████████████████| 364/364 [14:34<00:00,  2.40s/it]


크롤링한 데이터를 1617.csv 파일에 저장했습니다.
